In [7]:
from visualize2 import ResultParser
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from geometry import City
%matplotlib inline
from ipywidgets import fixed,SelectionSlider
from copy import deepcopy

# collecting result data
rp1 = ResultParser('0831_base_fixed_ratio')
df1 = rp1.prepare_all_data(force=True)

rp2 = ResultParser('0831_base_fixed_taxis')
df2 = rp2.prepare_all_data(force=True)

In [16]:
def plotting_percentile(p,d,R):
    """
    Where are the taxis from the dataset row, that have less income than the p-th percentile?
    """
    
    if R=='fixed':
        df = deepcopy(df1)
        mode = 'd'
    elif d=='fixed':
        df = deepcopy(df2)
        mode = 'R'
    
    row = df[df[mode] == eval(mode)].iloc[0]
    thres = np.percentile(row['trip_avg_price'],p)
    position = [row['position'].iloc[i] for i,price in enumerate(row['trip_avg_price']) if price<thres]
    
    
    print("TAXI POSITIONS AT END OF SIMULATION")
    
    fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(15,5))
    # taxis
    ax[0].hist2d(
        [x[0] for x in position],[x[1] for x in position],
        range = [[0,row['n']],[0,row['m']]],
        bins = [row['n'],row['m']],
        cmap = 'magma'
    )
    ax[0].axis('equal')
    ax[0].set_xlim(0,row['n'])
    ax[0].set_ylim(0,row['m'])
    ax[0].set_title('Taxis below the %dth percentile' % p)
    
    if 'request_destination_distributions' in row.index:
        if not hasattr(row['request_destination_distributions'],'dtype'):
            del row['request_destination_distributions']
        
    # request origin and destination distributions
    C = City(**row)
    
    o = []
    d = []
    for i in range(100000):
        ox,oy,dx,dy = C.create_one_request_coord()
        o.append([ox,oy])
        d.append([dx,dy])

    ax[1].hist2d(np.array(o)[:,0],np.array(o)[:,1],
        range = [[0,row['n']],[0,row['m']]],
        bins = [row['n'],row['m']]
    )
    ax[1].axis('equal')
    ax[1].set_xlim(0,row['n'])
    ax[1].set_ylim(0,row['m'])
    ax[1].set_title('Request origin distribution, R=%.2f, d=%d' % (row['R'],row['d']))
              
    plt.hist2d(np.array(d)[:,0],np.array(d)[:,1],
        range = [[0,row['n']],[0,row['m']]],
        bins = [row['n'],row['m']]
    )
    ax[2].axis('equal')
    ax[2].set_xlim(0,row['n'])
    ax[2].set_ylim(0,row['m'])
    ax[2].set_title('Request destination distribution, R=%.2f, d=%d' % (row['R'],row['d']))
    plt.show()
    
    print("INCOME VS TAXI ")
    
    fig,axn = plt.subplots(nrows=1,ncols=4,figsize=(20,5))
    for i,ratio in enumerate(['ratio_cruising','ratio_waiting', 'ratio_serving', 'ratio_to_request']):
        axn[i].plot(row['trip_avg_price'],row[ratio],'bo')
        axn[i].set_title(ratio)
        axn[i].set_ylim(0,1)
    plt.show()


In [17]:
interact(plotting_percentile,
         p = IntSlider(min=0,max=100, value=100,step=5),
         R = fixed('fixed'),
         d = SelectionSlider(value = 100, options=sorted(df1['d'].tolist())))

interactive(children=(IntSlider(value=100, description='p', step=5), SelectionSlider(description='d', options=(100.0, 150.0, 250.0, 400.0, 350.0, 300.0, 200.0), value=100.0), Output()), _dom_classes=('widget-interact',))

<function __main__.plotting_percentile>

In [20]:
interact(plotting_percentile,
         p = IntSlider(min=0,max=100, value=100,step=5),
         R = SelectionSlider(value = 0.05, options=sorted(df2['R'].tolist())),
         d = fixed('fixed'))

interactive(children=(IntSlider(value=100, description='p', step=5), SelectionSlider(description='R', index=1, options=(0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0), value=0.05), Output()), _dom_classes=('widget-interact',))

<function __main__.plotting_percentile>